In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.size'] = 14

In [ ]:

#sim_Boris_0_01 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_0_01/Boris_0_01.in")

sim_Boris_0_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_0_1/Boris_0_1.in")

sim_Boris_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_1/Boris_1.in")

sim_Boris_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_10/Boris_10.in")


sim_GcaCorr_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/GcaCorr/GcaCorr_1/GcaCorr_1.in")

sim_GcaCorr_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/GcaCorr/GcaCorr_10/GcaCorr_10.in")

In [ ]:
def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj


def plot_distance(sim, sim_baseline, label, particles, tmin = 0, tmax = 100000000000, step=1, unload=False, fig=None, ax=None):

    dist=[]
    for particle in particles:
        traj = get_trajectory(sim, particle, tmin, tmax)
        traj_baseline = get_trajectory(sim_baseline, particle, tmin, tmax)

        x1 = traj[0, :] - traj_baseline[0, ::step]
        x2 = traj[1, :] - traj_baseline[1, ::step]
        x3 = traj[2, :] - traj_baseline[2, ::step]

        dist.append(np.sqrt(x1**2 + x2**2 + x3**2))

    t = traj[3, :]
    if unload:
        sim["test_electrons"]["tracks"].unload()
        sim_baseline["test_electrons"]["tracks"].unload()
        
    #average for every particle
    avrg_dist = np.mean(np.array(dist), axis=0)

    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    if ax is None:
        ax = fig.add_subplot(111)

    ax.plot(t, avrg_dist, label=label)

    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
    ax.set_ylabel('mean dist to baseline' + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]))
    ax.legend()

    return fig, ax



def plot_trajectory(sim, label, particle, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):

    traj = get_trajectory(sim, particle, tmin, tmax, unload)
    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        label2 = ("$t = {:.3f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))


    if label == "Baseline":
        color = "black"
        markersize = 0
    else:
        color = None
        markersize = 1.5
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', markersize=markersize, linewidth=0.8, label=label, color=color)
    
    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]))
    ax.set_ylabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x2"]))
    ax.set_zlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x3"]))
    ax.legend()

    return fig, ax

In [ ]:
# Boris pusher trajectories
particle = range(0, 1000)
tmax = 700
baseline = sim_Boris_0_1

fig, ax = None, None                            
fig, ax = plot_distance(sim_Boris_10, baseline, r"Boris, $2\pi dt w_{ce} = 10$", particles=particle, tmax=tmax, step=100, fig=fig, ax=ax)
fig, ax = plot_distance(sim_Boris_1, baseline, r"Boris, $2\pi dt w_{ce} = 1$", particles=particle, tmax=tmax, step=10, fig=fig, ax=ax)

fig, ax = plot_distance(sim_GcaCorr_10, baseline, r"GcaCorr, $2\pi dt w_{ce} = 10$", particles=particle, tmax=tmax, step=100, fig=fig, ax=ax)
fig, ax = plot_distance(sim_GcaCorr_1, baseline, r"GcaCorr, $2\pi dt w_{ce} = 1$", particles=particle, tmax=tmax, step=10, fig=fig, ax=ax)

ax.set_title("Particles " + str(particle))
plt.tight_layout()
plt.show()

In [ ]:
# Boris pusher trajectories
particle = 5
tmax = 700

for particle in range(0, 1000, 200):
    
    fig, ax = None, None                            
    fig, ax = plot_trajectory(sim_Boris_0_1, r"Boris, $2\pi dt w_{ce} = 0.1$", particle=particle, tmax=tmax, fig=fig, ax=ax)
    fig, ax = plot_trajectory(sim_Boris_1, r"Boris, $2\pi dt w_{ce} = 1$", particle=particle, tmax=tmax, fig=fig, ax=ax)
    fig, ax = plot_trajectory(sim_Boris_10, r"Boris, $2\pi dt w_{ce} = 10$", particle=particle, tmax=tmax, fig=fig, ax=ax)

    fig, ax = plot_trajectory(sim_GcaCorr_1, r"GcaCorr, $2\pi dt w_{ce} = 1$", particle=particle, tmax=tmax, fig=fig, ax=ax)
    fig, ax = plot_trajectory(sim_GcaCorr_10, r"GcaCorr, $2\pi dt w_{ce} = 10$", particle=particle, tmax=tmax, fig=fig, ax=ax)

    ax.xaxis.labelpad = 10
    ax.yaxis.labelpad = 10
    ax.zaxis.labelpad = 10
    ax.set_title("Particle " + str(particle))
    ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5), fontsize='small')
    #plt.subplots_adjust(right=0.75)
    fig.tight_layout()
    plt.show()

